Attempting to scrape information from a webpage whose output changes dynamically as we enter text in a field

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import re
import time

Atttempting to get rid of initial popup:

In [11]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://webpricer.cms.gov/#/provider-directory/ipps')

driver.find_element(by = By.XPATH, value = "/html/body/div[3]/div/div/div/div/aside/a[2]").send_keys(Keys.RETURN)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


Experimenting to find the right web element and regex used to extract the data:

In [13]:
ID = "100030"

provider_list = [ID]

In [14]:
driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").send_keys(ID)

In [ ]:
table1 = driver.find_elements(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[1]/table/tbody")[0].text
table2 = driver.find_elements(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[2]/table/tbody")[0].text
table = table1 + "\n" + table2
print(table)


In [10]:
# print(table.split("\n"))
print(len(table.split("\n")))
column_names = ["ProviderID"]
for i in table.split("\n"):
    column_names.append(i[:i.rfind(' ')])
# print(column_names)
for i in range(len(column_names)):
    column_names[i] = column_names[i].replace(' ','_')
print(column_names)

60
['ProviderID', 'Fiscal_year_begin_date', 'Report_date', 'Termination_Date', 'Waiver_code', 'Intermediary_number', 'Provider_type', 'Current_Census_division', 'Change_code_for_reclass', 'Actual_Geographic_MSA', 'Wage_Index_MSA', 'Standardized_amount_MSA', 'Sole_community_or_Medicare', 'LUGAR', 'Temporary_relief_indicator', 'Federal_PPS_Blend_indicator', 'Facility_specific_rate', 'Cost_of_living_adjustment', 'Intern_to_bed_ratio', 'Bed_size', 'Cost-to-charge_ratio', 'Case_mix_index', 'SSI_ratio', 'Medicaid_ratio', 'Provider_PPS_period', 'Special_provider_update_factor', 'Operating_DSH_percent', 'Fiscal_year_end_date', 'Special_wage_index', 'Special_payment_indicator', 'Hospital_quality_indicator', 'Geographic_location_CBSA', 'Reclassification_CBSA', 'Standardized_amount_CBSA', 'Total_pass_through_amount_and_miscellaneous', 'Pass_through_amount_capital', 'Pass_through_amount_direct', 'Pass_through_amount_organ', 'Capital_PPS_pay_code', 'Capital_hospital_specific_rate', 'Capital_old_har

In [37]:
provider_list = [ID]
for i in table.split("\n"):
    provider_list.append(i[i.rfind(' ')+1:])
print(len(provider_list))

61


Importing the inpatient provider numbers and checking to see if it laoded correctly:

In [5]:
inpatient = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Medicare CCN Inpatient Facility.csv")
inpatient.head(5)

,Inpatient,CCN
0,NaN,10001
1,NaN,10005
2,NaN,10006
3,NaN,10007
4,NaN,10008


Run this block to perform web scraping:

In [12]:

inpatient = pd.read_csv(r"C:\Users\email\OneDrive\Documents\Python\Web-Scraping\Medicare CCN Inpatient Facility.csv")

try:
    driver.close()
except Exception:
    pass

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://webpricer.cms.gov/#/provider-directory/ipps')
driver.find_element(by = By.XPATH, value = "/html/body/div[3]/div/div/div/div/aside/a[2]").send_keys(Keys.RETURN)

ID_list = inpatient["CCN"].to_list()
ID_list = [str(x) for x in ID_list]
for i in range(568):
    ID_list[i] = "0" + ID_list[i]
     
final_list = []
df = pd.DataFrame([])

for ID in ID_list:
    driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").send_keys(ID)
    time.sleep(1)
    while True:
        try: 
            table1 = driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[1]/table/tbody").text
            break;
        except Exception:
            time.sleep(3)
    while True:
        try:
            table2 = driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[4]/main/table/tbody/tr/td[2]/table/tbody").text
            break;
        except Exception:
            time.sleep(3)
    table = table1 + "\n" + table2
    provider_list = [ID]
    for i in table.split("\n"):
        provider_list.append(i[i.rfind(' ')+1:])
    final_list.append(provider_list)
    driver.find_element(by = By.XPATH, value = "/html/body/div[1]/section/section[2]/div[2]/div[2]/div[3]/div[2]/input").clear()
    print(ID + " is done!", end = "\r")

df = pd.DataFrame(final_list, columns= column_names)
df.head(10)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\email\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


ValueError: 61 columns passed, passed data had 91 columns

In [18]:
df = pd.DataFrame(final_list, columns= column_names)
df.head(10)

,ProviderID,Fiscal_year_begin_date,Report_date,Termination_Date,Waiver_code,Intermediary_number,Provider_type,Current_Census_division,Change_code_for_reclass,Actual_Geographic_MSA,...,Hospital_readmission_reduction_adjustment,Model_1_bundle_percent,Hospital_acquired_condition_(HAC)_reduction_program_participant,Uncompensated_care_amount,Electronic_health_record_(EHR)_incentive_program_reduction_indicator,Low-volume_adjustment_factor,County_code,State_code,Medicare_performance_adjustment,LTCH_DPP_indicator
0,010001,10/01/2021,03/31/2022,--,N,10111,07,5,Y,2180,...,0.9903,0,N,$672.33,--,0,01069,01,0,--
1,010005,10/01/2021,03/31/2022,--,N,10111,07,5,Y,01,...,0.9985,0,Y,$912.95,--,0,01095,01,0,--
2,010006,07/01/2021,03/31/2022,--,N,10111,07,5,N,2650,...,0.984,0,N,$327.96,--,0,01077,01,0,--
3,010007,10/01/2021,03/31/2022,--,N,10111,14,5,--,01,...,0.9949,0,N,$503.44,--,0.201364,01039,01,0,--
4,010008,01/01/2022,03/31/2022,--,N,10111,00,5,--,01,...,1,0,N,$58.17,--,0.25,01041,01,0,--
5,010011,07/01/2021,03/31/2022,--,N,10111,00,5,--,1000,...,0.9905,0,N,"$1,471.06",--,0,01073,01,0,--
6,010012,04/01/2021,03/31/2022,--,N,05901,16,5,--,01,...,0.9953,0,N,$554.16,--,0.078712,01049,01,0,--
7,010016,01/01/2022,03/31/2022,--,N,10111,00,5,N,1000,...,0.998,0,N,$950.22,--,0,01117,01,0,--
8,010019,10/01/2021,03/31/2022,--,N,10111,00,5,N,2650,...,0.9758,0,N,$554.69,--,0,01033,01,0,--
9,010021,10/01/2021,03/31/2022,--,N,10111,00,5,--,01,...,0.9974,0,Y,"$1,426.64",--,0.195379,01045,01,0,--


In [17]:
for i in range(len(final_list)):
    if len(final_list[i]) != 61:
        final_list[i] = final_list[i][:61] 
        print(final_list[i])



['050137', '01/01/2022', '03/31/2022', '--', 'N', '01911', '07', '9', '--', '0000', '0000', '0000', '--', '--', '--', '--', '$2,788.67', '1', '0.0114', '183', '0.338', '1.3803', '0.0871', '0.1659', '--', '--', '--', '12/31/2016', '--', 'Y', '1', '31084', '37100', '05', '$3.78', '--', '$3.78', '--', 'C', '384.57', '0', '0', '0.008', '--', '0.0233', '0', '--', '--', 'N', '0', '1', '1', '0', 'N', '$4,029.72', '--', '0', '06037', '05', '0', '--']


In [16]:
x = [1,2,3,4,5]
print(x[:3])

[1, 2, 3]


In [21]:
df.to_csv("test.csv")

In [20]:
print(final_list[0])

['010001', '10/01/2021', '03/31/2022', '--', 'N', '10111', '07', '5', 'Y', '2180', '01', '01', '--', '--', '--', '--', '$1,918.13', '1', '0.0494', '322', '0.181', '--', '0.1029', '0.1989', '--', '--', '--', '09/30/2022', '0.856', '2', '1', '20020', '18880', '01', '$106.82', '--', '$46.13', '--', 'C', '0', '0', '0', '0.012', '--', '0.1136', '0', '--', '--', 'N', '0', '1', '0.9903', '0', 'N', '$672.33', '--', '0', '01069', '01', '0', '--']
